# Import Packages

In [1]:
import pandas as pd
import numpy as np


# Importing Data


In [4]:
import wrds
db=wrds.Connection()
compustat=db.raw_sql('''
select cik,gvkey, tic, fyear,datadate,clo,
mkvalt,at,ceq,lt,ib,csho,prcc_f,revt,sich
From comp.funda WHERE fyear>2010
and (indfmt='INDL') and (datafmt='STD') and (popsrc='D') and (consol='C')
''')

Enter your WRDS username [kstan]:krs84
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


# Displaying data


In [5]:
compustat[:10]
#Or
compustat.head(10)


,cik,gvkey,tic,fyear,datadate,clo,mkvalt,at,ceq,lt,ib,csho,prcc_f,revt,sich
0,0000002230,001119,ADX,2012.0,2012-12-31,None,NaN,NaN,NaN,NaN,NaN,91.106,10.59,NaN,NaN
1,0000002230,001119,ADX,2013.0,2013-12-31,None,NaN,NaN,NaN,NaN,NaN,92.216,13.07,NaN,NaN
2,0000002230,001119,ADX,2014.0,2014-12-31,None,NaN,NaN,NaN,NaN,NaN,93.509,13.68,NaN,NaN
3,0000001750,001004,AIR,2011.0,2012-05-31,None,485.2897,2195.653,864.649,1329.631,67.723,40.273,12.05,2074.498,5080.0
4,0000001750,001004,AIR,2012.0,2013-05-31,None,790.0029,2136.900,918.600,1217.400,55.000,39.382,20.06,2167.100,5080.0
5,0000001750,001004,AIR,2013.0,2014-05-31,None,961.3080,2199.500,999.500,1198.800,72.900,39.560,24.30,2035.000,5080.0
6,0000001750,001004,AIR,2014.0,2015-05-31,None,1046.3954,1515.000,845.100,669.900,-54.500,35.423,29.54,1594.300,5080.0
7,0000001750,001004,AIR,2015.0,2016-05-31,None,842.5112,1442.100,865.800,576.300,40.500,34.515,24.41,1662.600,5080.0
8,0000001750,001004,AIR,2016.0,2017-05-31,None,1200.3288,1504.100,914.200,589.900,50.200,34.354,34.94,1767.600,5080.0
9,0000002230,001119,ADX,2015.0,2015-12-31,None,NaN,NaN,NaN,NaN,NaN,95.420,12.83,NaN,NaN


# Keep Data with only certain characteristics


In [6]:

#One Variable:
a=compustat[compustat['at']>0]

#Multiple Variables (use | for or and use & for and):
a=compustat[(compustat['at']>0)&compustat['ceq'].notna()]



# Lag variables


In [7]:
compustat['atlag']=compustat.groupby('gvkey')['at'].shift(1)


Note that the groupby is the group you are lagging within. This is equivalent to bysort shift in Stata. Unfortunately, I am not aware of any equivalent to xtset and then L.variable in python, but this gets pretty close.


# Generating an indicator variable


In [8]:
compustat['at>5']=(compustat['at']>5).astype(int)


# Random Sampling


In [9]:
compustat2=compustat.sample(100)


# Simple Merging


Note that here you can specify left_on, right_on, and how. Simple merges are fast but are limited in that they cannot merge between things (i.e. dates)


In [10]:
compustat3=pd.merge(compustat,compustat2,on=['gvkey','datadate'])


# Pull in the linktable


I have learned coding over the years from looking at a lot of people’s stuff. Sometimes I have forgotten who I have used the code from (usually a lot of anonymous stack overflow pages), but will always try and recognize where I remember. Thanks to Freda Song Drechsler for this one.

In [11]:
query2=""" select gvkey as gvkey2, lpermco as permco, lpermno as permno, linkdt, linkenddt
from crsp.ccmxpf_linktable
where usedflag=1 and linkprim in ('P', 'C')"""
link=db.raw_sql(query2)


# SQL Style Merge


Note that here you establish a database, input the desired databases into the query, and then run the sql query.


In [12]:

import sqlite3
conn = sqlite3.connect(':memory:')
compustat.to_sql('comp', conn, index=False)
link.to_sql('link', conn, index=False)

qry = '''
select
*
from
comp join link on comp.gvkey=link.gvkey2
and
(((comp.datadate >= link.linkdt) or (link.linkdt is null)) and ((comp.datadate <= link.linkenddt) or (link.linkenddt is null)))
'''
comp = pd.read_sql_query(qry, conn)

# Conditional Replace

In [13]:
mask = compustat['at'] < 5
column_name = 'revt'
compustat.loc[mask, column_name] = np.nan


# Convert to Numeric

In [14]:
#Requires the given format
compustat['gvkey']=compustat['gvkey'].astype(int)
#drops non-numeric
compustat['at']=pd.to_numeric(compustat['at'],errors='coerce')

# Rename a column

In [15]:
compustat.rename(columns={'at':'awesome'}).head()


,cik,gvkey,tic,fyear,datadate,clo,mkvalt,awesome,ceq,lt,ib,csho,prcc_f,revt,sich,atlag,at>5
0,0000002230,1119,ADX,2012.0,2012-12-31,None,NaN,NaN,NaN,NaN,NaN,91.106,10.59,NaN,NaN,NaN,0
1,0000002230,1119,ADX,2013.0,2013-12-31,None,NaN,NaN,NaN,NaN,NaN,92.216,13.07,NaN,NaN,NaN,0
2,0000002230,1119,ADX,2014.0,2014-12-31,None,NaN,NaN,NaN,NaN,NaN,93.509,13.68,NaN,NaN,NaN,0
3,0000001750,1004,AIR,2011.0,2012-05-31,None,485.2897,2195.653,864.649,1329.631,67.723,40.273,12.05,2074.498,5080.0,NaN,1
4,0000001750,1004,AIR,2012.0,2013-05-31,None,790.0029,2136.900,918.600,1217.400,55.000,39.382,20.06,2167.100,5080.0,2195.653,1


# Require a minnimum number of observations


In [16]:

a=compustat.groupby('gvkey')['datadate'].count().reset_index()
compustat=pd.merge(compustat,a.rename(columns={'datadate':'count'}), on=['gvkey'])
compustat3=compustat[compustat['count']>7].reset_index(drop=True)


# Format as date


In [17]:
compustat['datadate']=pd.to_datetime(compustat['datadate'])


# Replace 0 with missing


In [18]:
lis=['at','revt']
for x in lis:
    mask = compustat[x] == 0
    column_name = x
    compustat.loc[mask, column_name] = np.nan


# Fill in a missing value


In [19]:
compustat['at']=compustat['at'].fillna(compustat['at'].mean())


# Drop missing columns


In [20]:
compustat=compustat.drop('clo',1)


# Drop missing values


In [21]:
compustat=compustat.dropna(subset=['cik']).reset_index(drop=True)


# Basic Descriptives


In [22]:
compustat.describe()


,gvkey,fyear,mkvalt,at,ceq,lt,ib,csho,prcc_f,revt,sich,atlag,at>5,count
count,71133.000000,71133.000000,5.144200e+04,7.113300e+04,64354.000000,6.439900e+04,64228.000000,67589.000000,62751.000000,54008.000000,56719.000000,5.389100e+04,71133.000000,71133.000000
mean,83607.209270,2014.644708,4.470397e+03,1.527321e+04,2497.734389,1.264967e+04,268.003811,194.545579,31.529576,4622.991619,4767.270033,1.596839e+04,0.816794,7.481605
std,75103.895215,2.469079,2.361381e+04,1.106384e+05,11785.929129,1.083876e+05,1656.125729,969.431497,301.679146,18914.526672,2075.535457,1.190229e+05,0.386838,2.032123
min,1004.000000,2011.000000,0.000000e+00,1.000000e-03,-139965.000000,0.000000e+00,-22348.000000,0.000000,0.000001,-6749.631000,100.000000,0.000000e+00,0.000000,1.000000
25%,18636.000000,2012.000000,4.076105e+01,7.382000e+01,13.492750,1.680100e+01,-7.025250,16.165000,2.320000,59.725750,2870.000000,5.552850e+01,1.000000,7.000000
50%,35305.000000,2015.000000,2.825129e+02,8.011610e+02,156.313500,2.716150e+02,3.518500,43.800000,12.040000,397.633000,4899.000000,5.650570e+02,1.000000,8.000000
75%,161069.000000,2017.000000,1.798656e+03,6.597800e+03,977.672750,2.052391e+03,85.379000,115.000000,28.650000,2112.739250,6331.000000,3.339677e+03,1.000000,9.000000
max,331856.000000,2019.000000,1.073391e+06,3.503319e+06,424791.000000,3.488711e+06,83963.000000,100092.395000,30150.000000,521426.000000,9998.000000,3.418318e+06,1.000000,9.000000


# Transpose Stuff


In [23]:
np.transpose(compustat)


,0,1,2,3,4,5,6,7,8,9,...,71123,71124,71125,71126,71127,71128,71129,71130,71131,71132
cik,0000002230,0000002230,0000002230,0000002230,0000002230,0000002230,0000002230,0000002230,0000002230,0000001750,...,0001739445,0001739445,0001739445,0001739445,0001739445,0001720161,0001720161,0001720161,0001280776,0001280776
gvkey,1119,1119,1119,1119,1119,1119,1119,1119,1119,1004,...,328795,328795,328795,328795,328795,330227,330227,330227,331856,331856
tic,ADX,ADX,ADX,ADX,ADX,ADX,ADX,ADX,ADX,AIR,...,ACA,ACA,ACA,ACA,ACA,CTRM,CTRM,CTRM,IMUX,IMUX
fyear,2012,2013,2014,2015,2016,2017,2018,2019,2011,2011,...,2015,2016,2017,2018,2019,2017,2018,2019,2018,2019
datadate,2012-12-31 00:00:00,2013-12-31 00:00:00,2014-12-31 00:00:00,2015-12-31 00:00:00,2016-12-31 00:00:00,2017-12-31 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00,2011-12-31 00:00:00,2012-05-31 00:00:00,...,2015-12-31 00:00:00,2016-12-31 00:00:00,2017-12-31 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00,2017-09-30 00:00:00,2018-09-30 00:00:00,2019-12-31 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00
mkvalt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,485.29,...,NaN,NaN,NaN,1348.5,2151.76,NaN,NaN,6.1383,NaN,104.227
at,13667.6,13667.6,13667.6,13667.6,13667.6,13667.6,13667.6,13667.6,13667.6,2195.65,...,13667.6,1526.3,1602.5,2172.2,2302.5,8.718,9.624,30.421,13667.6,65.955
ceq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,864.649,...,NaN,1341.8,1407.9,1684.5,1790.4,8.494,9.475,13.204,NaN,58.363
lt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1329.63,...,NaN,184.5,194.6,487.7,512.1,0.224,0.149,17.217,NaN,7.592
ib,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.723,...,NaN,123,89.7,75.7,113.3,0.879,0.981,1.088,NaN,-34.933


# Basic Abnormal Returns


In [24]:
crsp=db.raw_sql("""select a.permno, a.permco, a.date,
b.shrcd, b.exchcd, b.siccd,
a.ret, a.vol, a.shrout, a.prc,a.cfacpr, a.cfacshr
from crsp.msf as a
left join crsp.msenames as b
on a.permno=b.permno
and b.namedt<=a.date
and a.date<=b.nameendt
where a.date between '01/01/2017' and '12/31/2018'
and b.shrcd between 10 and 11""")
crsp2=db.raw_sql("""select date, vwretd
from msi
""")
crsp2['vwretd']=np.log(1+crsp2['vwretd'])
crsp2['vwretsum']=crsp2['vwretd'].rolling(12).sum().reset_index(drop=True)
crsp=pd.merge(crsp,crsp2,on='date')
crsp=crsp.sort_values(['permno','date']).reset_index(drop=True)
crsp['logret']=np.log(1+crsp['ret'])
crsp['retsum']=crsp.groupby('permno')['logret'].rolling(12).sum().reset_index(drop=True)
crsp['aret']=np.exp(crsp['retsum'])-np.exp(crsp['vwretsum'])


C:\Users\kstan\Anaconda3\lib\site-packages\pandas\core\series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Form Quantiles


In [25]:
crsp['quartile']=pd.qcut(crsp['aret'],4,labels=False)


# Rolling Average


In [26]:
crsp2['avgvwretd']=crsp2['vwretd'].rolling(3).mean()


# Min Max Date By Group


In [27]:

a=compustat.groupby('gvkey')['datadate'].max().reset_index().rename(columns={'datadate':'maxdate'})
compustat=pd.merge(compustat,a,on='gvkey')
